# Méthode de Monte-Carlo pour déterminer le coefficient directeur moyen d'une droite de régression linéaire et son incertitude-type
### Détermination de l'intensité de pesanteur terrestre à l'aide d'un smartphone :

   - Mesure de la durée de chute d'un objet métallique, pour une hauteur $h$ donnée (utilisation du chronométre sonore de l'application phyphox);
   - pour chaque hauteur une dizaine de mesures sont réalisées, ce qui permet de calculer la durée moyenne $\Delta t$ (associée à chaque valeur de $h$);
   - l'incertitude-type de $h$, $u(h)$, est estimée par l'expérimentateur;
   - l'incertitude-type de $\Delta t$, $u(\Delta t)$, est évaluée par une méthode statistique (évaluation de type A).

Première cellule pour déclarer les bibliothèques à importer :

In [ ]:
#Importation des bibliothèques
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Affichage avec la bibliothèque graphique intégrée à Notebook
%matplotlib inline

Définition de la fonction *Alea* qui renvoie une valeur aléatoire de la variable $L[0]$ d'incertitude-type $L[1]$ :


In [23]:
def Alea(L):
    tirage=np.random.normal()   # Tirage entre -infini et +infini (loi normale)
    return L[0]+L[1]*tirage

La cellule ci-dessous est à compléter en précisant :
- les hauteurs de chutes mesurées
- l'évaluation de l'incertitude-type sur la mesure de $h$
- les durées moyennes de chute pour chaque hauteur
- l'évalution de l'incertitude sur $\Delta t$ par une méthode statistique

In [ ]:
h=[...]         #Hauteurs de chute (en m)
u_h=...         #Estimation de l'incertitude-type de h (en m)

Delta_t=[...]   #Durée moyenne de la chute (en s)
u_Delta_t=...   #Incertitude-type de Delta_t (en s) - Evaluation de type A

In [ ]:
Abs=[]
for k in range(len(Delta_t)):
    Abs.append(0.5*Delta_t[k]**2)

#Formule de propagation des incertitudes appliquée à Abs = 0.5*Delta_t^2
u_Abs=[]
for k in range(len(Delta_t)):
    u_Abs.append(Abs[k]*2*u_Delta_t/Delta_t[k])

#Préparation des listes avec incertitudes
hauteur=[]
for k in range(len(h)):
    hauteur.append([h[k],u_h])

duree=[]
for k in range(len(h)):
    duree.append([Delta_t[k],u_Delta_t])

def lineaire(x,a):
    return a*x

#Méthode de Monte Carlo pour déterminer le coefficient directeur moyen de la droite de regression linéaire
LCoef=[] #Initialisation de la liste vide (coefficient directeur)
iteration=100000

for i in range(iteration):
    Alea_abs=[]
    Alea_h=[]
    for k in range(len(h)):
        Alea_abs.append(0.5*Alea(duree[k])**2)
        Alea_h.append(Alea(hauteur[k]))
    params, covar = curve_fit(lineaire,Alea_abs,Alea_h)
    coef=params[0]
    LCoef.append(coef)

MoyPente=np.mean(LCoef)
uPente=np.std(LCoef,ddof=1) #ddof=1 : déviation standard au sens statistique (divisé par racine carré de n-1)

print('Pente :',round(MoyPente,5),'m.s^-2')
print('u(Pente) :',round(uPente,5),'m.s^-2')

Les instructions qui suivent assurent et paramètrent le tracé du graphique (fenêtre 1) et de l'histogramme (fenêtre 2) :

**Avec Anaconda :** une fois complétées, exécuter l'intégralité des cellules : *Cell $\rightarrow$ Run All*.

**En ligne (my Binder):** une fois complétées, exécuter l'intégralité des cellules : *Run $\rightarrow$ Run all cells*.


In [ ]:
fenetre1 = plt.figure('Pour 100000 iterations')
graph = fenetre1.add_subplot(111)
graph.hist(LCoef, range=(min(LCoef),max(LCoef)), bins = 50, color = 'blue', edgecolor = 'black')
plt.xlabel(r'$Pente \ (m.s^{-2})$')
plt.ylabel('Effectif')

fenetre2 = plt.figure('Hauteurs de chute')
plt.plot(Abs,h,'m+',label="Points expérimentaux")
#Tracé des barres d'incertitudes
plt.errorbar(Abs,h,xerr=u_Abs,yerr=u_h,fmt='none',ecolor='blue')
#Tracé de la droite moyenne
X = np.array([min(Abs),max(Abs)])     #Pour tracer une droite, 2 points sont suffisants
plt.plot(X,MoyPente*X,'r-',label="Droite moyenne")
plt.xlabel(r'$1/2 \times {\Delta t}^2 \ (en \ s^2)$')
plt.ylabel(r'$h \ (en \ m)$')
plt.axis([min(Abs),max(Abs),min(h),max(h)])
plt.title (r'$h = f(1/2 \times {\Delta t}^2)$')
plt.text(min(Abs)+0.0050,min(h)+0.025,"Equation de la droite moyenne : h ="+str(round(MoyPente,4))+r'$\times 1/2 \times {\Delta t}^2$',bbox=dict(facecolor='red', alpha=0.5))
plt.legend()
plt.grid()

plt.show()